In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
csv_path = r"../../dataset/source_folder/csv/books.csv"
multi_csv_path = r"../../dataset/source_folder/csv/"
parquet_path = r"../../dataset/source_folder/parquet/books.parquet"
json_path = r"../../dataset/source_folder/json/books.json"
json_multiline_path = r"../../dataset/source_folder/json/books_multiline.json"

In [3]:
spark = SparkSession.builder.appName("dataframe extraction").getOrCreate()
spark

## Reading CSV file

In [4]:
df = spark.read\
    .format("csv")\
    .option("schema", "book_id integer, book_title string, book_cost float, book_type string, publ_id integer, book_edition string, book_discount integer, isbn string, publ_date as string")\
    .option("header", True)\
    .option("sep", ",")\
    .option("inferSchema", False)\
    .load(csv_path)
df.show(5, truncate=False)

+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|BOOK_ID|BOOK_TITLE                         |BOOK_COST|BOOK_TYPE|PUBL_ID|BOOK_EDITION|BOOK_DISCOUNT|ISBN          |PUBL_DATE|
+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|B0056  |DataBase Systems                   |935.27   |DB       |11     |Fifth       |10           |978-8131716250|01-May-08|
|B0059  |Database Systems                   |1003.67  |DB       |11     |Eighth      |10           |978-8185015583|01-Jan-02|
|B0063  |The Java TM Programming Language   |425      |PR       |11     |Third       |10           |978-8131702215|01-Jan-08|
|B0067  |The Java TM EE5 Tutorial           |725      |PR       |11     |NULL        |18           |978-8131714928|03-Nov-06|
|B0071  |The Complete Reference HTML & XHTML|540      |PR       |15     |Fourth      |16           |978-0070701946|08-

## Reading Parquet file

In [5]:
df = spark.read\
    .format("parquet")\
    .option("header", True)\
    .option("inferSchema", True)\
    .load(parquet_path)
df.show(5, truncate=False)

+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|BOOK_ID|BOOK_TITLE                         |BOOK_COST|BOOK_TYPE|PUBL_ID|BOOK_EDITION|BOOK_DISCOUNT|ISBN          |PUBL_DATE|
+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|B0056  |DataBase Systems                   |935.27   |DB       |11     |Fifth       |10           |978-8131716250|01-May-08|
|B0059  |Database Systems                   |1003.67  |DB       |11     |Eighth      |10           |978-8185015583|01-Jan-02|
|B0063  |The Java TM Programming Language   |425      |PR       |11     |Third       |10           |978-8131702215|01-Jan-08|
|B0067  |The Java TM EE5 Tutorial           |725      |PR       |11     |NULL        |18           |978-8131714928|03-Nov-06|
|B0071  |The Complete Reference HTML & XHTML|540      |PR       |15     |Fourth      |16           |978-0070701946|08-

## Reading JSON file

### Single line JSON

In [9]:
# Read multi line JSON file into dataframe
df = spark.read \
    .format('org.apache.spark.sql.json') \
    .load(json_path)
df.show(5, truncate=False)

+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+
|BOOK_COST|BOOK_DISCOUNT|BOOK_EDITION|BOOK_ID|BOOK_TITLE      |BOOK_TYPE|ISBN          |PUBL_DATE|PUBL_ID|
+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+
|935.27   |10           |Fifth       |B0056  |DataBase Systems|DB       |978-8131716250|01-May-08|11     |
+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+



### MultiLine JSON

In [6]:
# Read multi line JSON file into dataframe
df = spark.read \
    .format('org.apache.spark.sql.json') \
    .option("multiline", True) \
    .load(json_multiline_path)
df.show(5, truncate=False)

+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+
|BOOK_COST|BOOK_DISCOUNT|BOOK_EDITION|BOOK_ID|BOOK_TITLE      |BOOK_TYPE|ISBN          |PUBL_DATE|PUBL_ID|
+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+
|935.27   |10           |Fifth       |B0056  |DataBase Systems|DB       |978-8131716250|01-May-08|11     |
+---------+-------------+------------+-------+----------------+---------+--------------+---------+-------+



## Reading Kafka

In [8]:
# df = spark.readStream\
#     .format('kafka')\
#     .option('kafka.bootstrap.servers', "kafka_bootstrap_servers")\
#     .option('subscribe', "kafka_topic")\
#     .option('startingoffsets', 'latest')\
#     .load()

In [12]:
employee_file = "../../dataset/source_folder/json/employees.json"
# Read multi line JSON file into dataframe
employees = spark.read \
    .format('org.apache.spark.sql.json') \
    .option("multiline", True) \
    .load(employee_file)
employees.show(5, truncate=False)

+-----------+------+--------+------+
|dept       |emp_id|emp_name|salary|
+-----------+------+--------+------+
|HR         |1     |Alice   |70000 |
|Engineering|2     |Bob     |100000|
|Engineering|3     |Charlie |95000 |
|HR         |4     |Diana   |75000 |
+-----------+------+--------+------+



In [18]:
from pyspark.sql.functions import sum, col
employees.groupBy("dept").agg(sum(col("salary")).alias("department_count")).show()

+-----------+----------------+
|       dept|department_count|
+-----------+----------------+
|Engineering|          195000|
|         HR|          145000|
+-----------+----------------+

